In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install neo4j
%pip install json-repair

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

from dotenv import load_dotenv

load_dotenv()

/Users/larissakemicloud.com/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


False

In [4]:
import os

os.environ["NEO4J_URI"] 
os.environ["NEO4J_USERNAME"] 
os.environ["NEO4J_PASSWORD"] 

# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [5]:
# Carregar os conteúdos dos arquivos SQL fornecidos
file_paths = [
    '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupo.sql',
    '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupoUsuario.sql',
]

# Lendo os conteúdos dos arquivos SQL
sql_contents = []
for path in file_paths:
    with open(path, 'r') as file:
        sql_contents.append(file.read())

# Exibir o conteúdo do primeiro arquivo como exemplo
sql_contents[0][:1000]  # Exibir os primeiros 1000 caracteres do primeiro arquivo para análise

'\ufeffCREATE TABLE [dbo].[ControleDeAcessoGrupo] (\n    [ID]                      INT            IDENTITY (1, 1) NOT NULL,\n    [ID_Empresa]              INT            NOT NULL,\n    [Nome]                    NVARCHAR (250) NOT NULL,\n    [Descricao]               NVARCHAR (600) NULL,\n    [Tipo]                    NVARCHAR (100) NOT NULL,\n    [Bloqueado]               BIT            NOT NULL,\n    [DataDaUltimaAlteracao]   DATETIME       NOT NULL,\n    [UltimoUsuarioQueAlterou] INT            NOT NULL,\n    [DataDeDelecao]           DATETIME       NULL,\n    CONSTRAINT [PK_ControleDeAcessoGrupo] PRIMARY KEY CLUSTERED ([ID] ASC)\n);\n\n'

In [6]:
# Criando as consultas Cypher para todas as tabelas baseadas nos arquivos SQL

queries = []

# ControleDeAcessoGrupo
queries.append("""
MERGE (g:Grupo {id: 1, nome: 'NomeDoGrupo', descricao: 'DescricaoGrupo', tipo: 'TipoGrupo'})
""")

# ControleDeAcessoGrupoUsuario - Relaciona Grupos a Usuários
queries.append("""
MERGE (g:Grupo {id: 1})
MERGE (u:Usuario {id: 101, nome: 'NomeDoUsuario'})
MERGE (g)-[:CONTAINS]->(u)
""")

# ControleDeAcessoMenu - Relaciona Menus e Submenus
queries.append("""
MERGE (m:Menu {id: 1, nome: 'MenuPrincipal'})
MERGE (sub:Menu {id: 2, nome: 'SubMenu'})
MERGE (sub)-[:CHILD_OF]->(m)
""")

# ControleDeAcessoModulo e ControleDeAcessoModuloTela - Relaciona Módulos a Telas
queries.append("""
MERGE (mod:Modulo {id: 1, nome: 'ModuloA'})
MERGE (tela:Tela {id: 1, nome: 'TelaA'})
MERGE (mod)-[:INCLUDES]->(tela)
""")

# ControleDeAcessoPermissao - Define permissões
queries.append("""
MERGE (p:Permissao {id: 1, nome: 'PermissaoA'})
""")

# ControleDeAcessoPermissaoMenu - Relaciona Permissões com Menus
queries.append("""
MERGE (p:Permissao {id: 1})
MERGE (m:Menu {id: 1})
MERGE (p)-[:APPLIES_TO]->(m)
""")

# ControleDeAcessoRecurso - Define Recursos
queries.append("""
MERGE (r:Recurso {id: 1, nome: 'RecursoA'})
""")

# ControleDeAcessoTela - Define Telas
queries.append("""
MERGE (t:Tela {id: 1, nome: 'TelaA'})
""")

# ControleDeAcessoUsuario - Define Usuários
queries.append("""
MERGE (u:Usuario {id: 101, nome: 'NomeDoUsuario', email: 'usuario@exemplo.com'})
""")

queries

["\nMERGE (g:Grupo {id: 1, nome: 'NomeDoGrupo', descricao: 'DescricaoGrupo', tipo: 'TipoGrupo'})\n",
 "\nMERGE (g:Grupo {id: 1})\nMERGE (u:Usuario {id: 101, nome: 'NomeDoUsuario'})\nMERGE (g)-[:CONTAINS]->(u)\n",
 "\nMERGE (m:Menu {id: 1, nome: 'MenuPrincipal'})\nMERGE (sub:Menu {id: 2, nome: 'SubMenu'})\nMERGE (sub)-[:CHILD_OF]->(m)\n",
 "\nMERGE (mod:Modulo {id: 1, nome: 'ModuloA'})\nMERGE (tela:Tela {id: 1, nome: 'TelaA'})\nMERGE (mod)-[:INCLUDES]->(tela)\n",
 "\nMERGE (p:Permissao {id: 1, nome: 'PermissaoA'})\n",
 '\nMERGE (p:Permissao {id: 1})\nMERGE (m:Menu {id: 1})\nMERGE (p)-[:APPLIES_TO]->(m)\n',
 "\nMERGE (r:Recurso {id: 1, nome: 'RecursoA'})\n",
 "\nMERGE (t:Tela {id: 1, nome: 'TelaA'})\n",
 "\nMERGE (u:Usuario {id: 101, nome: 'NomeDoUsuario', email: 'usuario@exemplo.com'})\n"]

In [7]:
# Executando as queries geradas no seu banco de dados Neo4j
queries = [
    """
    MERGE (g:Grupo {id: 1, nome: 'NomeDoGrupo', descricao: 'DescricaoGrupo', tipo: 'TipoGrupo'})
    """,
    """
    MERGE (g:Grupo {id: 1})
    MERGE (u:Usuario {id: 101, nome: 'NomeDoUsuario'})
    MERGE (g)-[:CONTAINS]->(u)
    """,
    """
    MERGE (m:Menu {id: 1, nome: 'MenuPrincipal'})
    MERGE (sub:Menu {id: 2, nome: 'SubMenu'})
    MERGE (sub)-[:CHILD_OF]->(m)
    """,
    """
    MERGE (mod:Modulo {id: 1, nome: 'ModuloA'})
    MERGE (tela:Tela {id: 1, nome: 'TelaA'})
    MERGE (mod)-[:INCLUDES]->(tela)
    """,
    """
    MERGE (p:Permissao {id: 1, nome: 'PermissaoA'})
    """,
    """
    MERGE (p:Permissao {id: 1})
    MERGE (m:Menu {id: 1})
    MERGE (p)-[:APPLIES_TO]->(m)
    """,
    """
    MERGE (r:Recurso {id: 1, nome: 'RecursoA'})
    """,
    """
    MERGE (t:Tela {id: 1, nome: 'TelaA'})
    """,
    """
    MERGE (u:Usuario {id: 101, nome: 'NomeDoUsuario', email: 'usuario@exemplo.com'})
    """
]

# Assumindo que você tenha um objeto `graph` já conectado ao seu banco de dados Neo4j
for query in queries:
    graph.query(query)

In [8]:
# Consultas para deletar todos os nós e relacionamentos criados anteriormente
delete_queries = [
    """
    MATCH (g:Grupo {id: 1})
    DETACH DELETE g
    """,
    """
    MATCH (u:Usuario {id: 101})
    DETACH DELETE u
    """,
    """
    MATCH (m:Menu {id: 1})
    DETACH DELETE m
    """,
    """
    MATCH (sub:Menu {id: 2})
    DETACH DELETE sub
    """,
    """
    MATCH (mod:Modulo {id: 1})
    DETACH DELETE mod
    """,
    """
    MATCH (tela:Tela {id: 1})
    DETACH DELETE tela
    """,
    """
    MATCH (p:Permissao {id: 1})
    DETACH DELETE p
    """,
    """
    MATCH (r:Recurso {id: 1})
    DETACH DELETE r
    """
]

# Assumindo que você tenha um objeto `graph` já conectado ao seu banco de dados Neo4j
for delete_query in delete_queries:
    graph.query(delete_query)

In [9]:
graph.query(
    """
    MATCH (u:Usuario {id: 101})-[r1:BELONGS_TO]->(e:Empresa {id: 1})
    MATCH (u)-[r2:HAS_TOKEN]->(t:Token {token: 'abc123token'})
    DETACH DELETE r1, r2, u, e, t
    """
)

[]